# ch 5.6 改进的BP算法
试设计一个BP改进算法，能通过动态调整学习率显著提升收敛速度。编程实现该算法，并选择两个UCI数据集与标准BP算法进行比较。

Note：
1. 学习率调整方法包括Adagrad，RMSProp等
2. 这里使用AdaGrad算法，算法如下（Ian Goodfellow《深度学习》P188）：
常数sigma=1e-7

梯度累积变量 r = 0

全局学习率 epsilon = 0.1

while 没有达到停止准则 do

    采集m个样本
    
    计算m个样本的平均梯度grad
    
    累积平方梯度r = r + grad * grad
    
    计算更新theta = theta - epsilon/(sigma+sqrt(r)) * grad

In [54]:
import numpy as np
import pandas as pd

In [59]:
dataset = pd.read_csv('data/blood.txt',header=None)
dataset.columns = ['x1','x2','x3','x4','label']
dataset = dataset.apply(lambda x:(x-np.min(x))/(np.max(x)-np.min(x)))
dataset_array = np.array(dataset)
dataset_array = dataset_array[np.random.permutation(len(dataset))]

In [55]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [56]:
def cal_acc(dataset_array, w_h_j, v_i_h, theta_j, gamma_h):
    dataset = dataset_array[:,:-1]
    label = dataset_array[:,-1]
    acc = 0
    for i in range(len(dataset)):
        pred_label = predict(dataset[i], w_h_j, v_i_h, theta_j, gamma_h)
        if pred_label == label[i]:
            acc += 1
    return acc/len(dataset)

In [57]:
def predict(x_i, w_h_j, v_i_h, theta_j, gamma_h):
    alpha_h = np.dot(x_i, v_i_h)
    b_h = sigmoid(alpha_h - gamma_h)
    beta_j = np.dot(b_h, w_h_j)        
    y_j_cap = sigmoid(beta_j - theta_j)
#     print(y_j_cap, end=" ")
    if y_j_cap[0][0] > y_j_cap[0][1]:
#         print(0)
        return 0
    else:
#         print(1)
        return 1


In [58]:
def adagrad_bp(dataset_array, niter):       
    h_num = 30
    j_num = 2
    i_num = dataset_array.shape[1] - 1
    l_num = 2
    epsilon = 0.1
    sigma = 10e-7
    m_num = 20
    w_h_j = np.random.random([h_num, j_num])
    v_i_h = np.random.random([i_num, h_num])
    theta_j = np.random.random([1,j_num])
    gamma_h = np.random.random([1,h_num])
    r_w_h_j = np.zeros([h_num, j_num])
    r_v_i_h = np.zeros([i_num, h_num])
    r_theta_j = np.zeros([1,j_num])
    r_gamma_h = np.zeros([1,h_num])
    error_list = []
    for i in range(niter):
        error = 0
        delta_w_h_j = np.zeros([h_num, j_num])
        delta_theta_j = np.zeros([1, j_num])
        delta_v_i_h = np.zeros([i_num, h_num])
        delta_gamma_h = np.zeros([1, h_num])
        for _ in range(m_num):
            k = np.random.randint(len(dataset_array))
            x_i, y_i = dataset_array[k,:-1], dataset_array[k,-1]  # x_i：1xi的向量
            x_i = np.array(x_i)
            x_i = np.reshape(x_i, [1,i_num])
            y_j = np.zeros([1,l_num])
            y_j[0][int(y_i)] = 1
            alpha_h = np.dot(x_i, v_i_h)
            b_h = sigmoid(alpha_h - gamma_h)
            beta_j = np.dot(b_h, w_h_j)        
            # formula 5.3
            y_j_cap = sigmoid(beta_j - theta_j)
            # formula 5.10
            g_j = y_j_cap * (np.ones_like(y_j_cap)-y_j_cap)*(y_j - y_j_cap)
            # formula 5.15
            e_h = b_h *(np.ones_like(b_h)-b_h) * np.dot(g_j, w_h_j.T)
            
            delta_w_h_j += np.dot(b_h.T, g_j)
            delta_theta_j += -g_j
            delta_v_i_h += (np.dot(e_h.T, x_i)).T
            delta_gamma_h += -e_h
            error += 0.5 * np.sum((y_j_cap - y_j) * (y_j_cap - y_j))

        delta_w_h_j /= m_num
        delta_theta_j /= m_num
        delta_v_i_h /= m_num
        delta_gamma_h /= m_num

        r_w_h_j += delta_w_h_j*delta_w_h_j
        r_v_i_h += delta_v_i_h*delta_v_i_h
        r_theta_j += delta_theta_j*delta_theta_j
        r_gamma_h += delta_gamma_h*delta_gamma_h

        w_alpha = np.ones_like(delta_w_h_j) * epsilon / (np.ones_like(delta_w_h_j) * sigma + np.sqrt(r_w_h_j))
        theta_alpha = np.ones_like(delta_theta_j)*epsilon/(np.ones_like(delta_theta_j)*sigma+np.sqrt(r_theta_j))
        v_alpha = np.ones_like(delta_v_i_h)*epsilon/(np.ones_like(delta_v_i_h)*sigma+np.sqrt(r_v_i_h))
        gamma_alpha = np.ones_like(delta_gamma_h)*epsilon/(np.ones_like(delta_gamma_h)*sigma+np.sqrt(r_gamma_h))

        #这里用+是因为delta里面其实是负梯度
        w_h_j += w_alpha*delta_w_h_j
        theta_j += theta_alpha*delta_theta_j
        v_i_h += v_alpha*delta_v_i_h
        gamma_h += gamma_alpha*delta_gamma_h


        if i % 10 == 0:
            error_list.append(error/m_num)
    print(error_list)
    return w_h_j, v_i_h, theta_j, gamma_h

In [67]:
%%time
test_num = dataset.shape[0]//5
w_h_j, v_i_h, theta_j, gamma_h = adagrad_bp(dataset_array[test_num:], niter =100)
acc = cal_acc(dataset_array[:test_num], w_h_j, v_i_h, theta_j, gamma_h)
print(acc)

[0.49580279793352455, 0.30749959061012194, 0.23420224929534378, 0.15867765393551489, 0.18506877978027941, 0.18003805913516061, 0.13009109482011777, 0.15181505713283036, 0.34733435432216569, 0.12839970876641127]
0.8120805369127517
Wall time: 135 ms


In [68]:
def standard_bp(dataset_array,niter =350, eta=0.1):    
    h_num = 10
    j_num = 2
    i_num = dataset_array.shape[1] - 1
    l_num = 2
    w_h_j = np.random.random([h_num, j_num])
    v_i_h = np.random.random([i_num, h_num])
    theta_j = np.random.random([1,j_num])
    gamma_h = np.random.random([1,h_num])
    error_list = []
    for i in range(niter):
        error = 0
        for k in range(len(dataset_array)):
            x_i, y_i = dataset_array[k,:-1], dataset_array[k,-1]  # x_i：1xi的向量
            x_i = np.array(x_i)
            x_i = np.reshape(x_i, [1,i_num])
            y_j = np.zeros([1,l_num])
            y_j[0][int(y_i)] = 1
            alpha_h = np.dot(x_i, v_i_h)
            b_h = sigmoid(alpha_h - gamma_h)
            beta_j = np.dot(b_h, w_h_j)        
            # formula 5.3
            y_j_cap = sigmoid(beta_j - theta_j)
            # formula 5.10
            g_j = y_j_cap * (np.ones_like(y_j_cap)-y_j_cap)*(y_j - y_j_cap)
            # formula 5.15
            e_h = b_h *(np.ones_like(b_h)-b_h) * np.dot(g_j, w_h_j.T)

            delta_w_h_j = eta * np.dot(b_h.T, g_j)
            delta_theta_j = -eta * g_j
            delta_v_i_h = eta * (np.dot(e_h.T, x_i)).T
            delta_gamma_h = -eta * e_h

            w_h_j += delta_w_h_j
            theta_j += delta_theta_j
            v_i_h += delta_v_i_h
            gamma_h += delta_gamma_h

            error += 0.5*np.sum((y_j_cap - y_j)*(y_j_cap - y_j))
        if i % 10 == 0:
            error_list.append(error/len(dataset_array))   
    print(error_list)
    return w_h_j, v_i_h, theta_j, gamma_h

In [69]:
%%time
test_num = dataset.shape[0]//5
w_h_j, v_i_h, theta_j, gamma_h = standard_bp(dataset_array[test_num:], niter =100, eta=0.1)
acc = cal_acc(dataset_array[:test_num], w_h_j, v_i_h, theta_j, gamma_h)
print(acc)

[0.21627199610317085, 0.1847302176018347, 0.17494199178601993, 0.16656512135352605, 0.16316608869214269, 0.16196825401081952, 0.16138577931454817, 0.16095433576224158, 0.16055459951763884, 0.16016020456760566]
0.8120805369127517
Wall time: 3.01 s
